# PyTorch 从零实现 Transformer

## 目录

1. Introduction
2. Basic Components
    1. Create Word Embeddings
    2. Positional Encoding
    3. Self Attention
3. Encoder
4. Decoder
5. Testing
6. Some useful resources

# 1.Introduction

本内容如何使用 Pytorch 从零开始实现 Transformer 架构，想要深入了解 Transformer 原理推荐下面内容：

1. [我自己写的笔记](https://github.com/CliffKai/DeepL-by-Pytorch/blob/master/Paper%20Reading/NLP/Transformer.md)
2. [Transformer 论文:Attention is All you Need](https://proceedings.neurips.cc/paper/2017/hash/3f5ee243547dee91fbd053c1c4a845aa-Abstract.html)
3. [李沐老师的d2l](https://zh-v2.d2l.ai/chapter_attention-mechanisms/index.html)
4. [Jay Alammar 的 Blog](https://jalammar.github.io/illustrated-transformer/)

![Figure_1](../../images/Transformer_from_scratch_Figure_1.png)

# 2.Basic Components

首先我们导入所需的库

In [7]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
print(torch.__version__)
if torch.cuda.is_available():
    print("CUDA is available")

2.5.1+cu124
CUDA is available


## 2.1 Create Word Embeddings

首先我们需要将输入序列中的每个词转化为一个嵌入向量，将离散的词语表示转换为具有语义意义的稠密向量表示，以便神经网络可以理解、处理和学习语言中的词语之间的关系。

这里我们假设每个嵌入向量的维度是 512，构建一个大小为 100 的词汇表，嵌入矩阵为 100 × 512。也就是说每个词会被编码为 512 维的一个向量，并且这个向量是可学习的。

举个例子：假设我们的 batch_size = 32，每个 batch_size 的长度为 10 个词，那我们的输出向量则为 32x10x512。

In [9]:
# 所有 PyTorch 的模型、层都需要继承自 nn.Module
class Embedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        """
        Args:
            vocab_size:词汇表大小
            embed_dim:词嵌入维度
        """
        super(Embedding, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)

    def forward(self, x):
        """
        Args:
            x:输入的词索引张量，shape 通常为 [batch_size, seq_len]
        Return:
            out:词向量张量，形状为 [batch_size, seq_len, embed_dim]
        """
        out = self.embed(x)
        return out

## 2.2 Positional Encoding

Transformer 自身无法感知顺序，所以需要位置编码来让模型知道每个词所在的位置。

此处使用原论文中的 Sinusoidal Positional Encoding（正余弦位置编码）。

In [11]:
class PositionalEmbedding(nn.Module):
    def __init__(self, max_seq_len, embed_model_dim):
        """
        Args:
            max_seq_len:最长支持的序列长度
            embed_model_dim:嵌入向量的维度
        """
        super(PositionalEmbedding, self).__init__()
        self.embed_dim = embed_model_dim

        pe = torch.zeros(max_seq_len, slef.embed_dim)
        for pos in range(max_seq_len):
            for i in range(0, self.embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/self.embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/self.embed_dim)))

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x:词嵌入后的张量，shape = [batch_size, seq_len, embed_dim]
        Return:
            x:加入了位置编码的词嵌入张量，shape = [batch_size, seq_len, embed_dim]
        """
        # 防止位置编码影响太大，提高训练稳定性
        x = x * math.sqrt(self.embed_dim)
        seq_len = x.size(1)
        # 使用 torch.autograd.Variable(..., requires_grad=False) 包裹是为了防止参与梯度计算
        x = x + torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)
        return x

## 2.3 Self Attention

![MultiHeadAttention](../../images/Transformer_from_scratch_Figure_MultiHeadAttention.png)

关于 Attention 的详细解释，请参照 Introduction 中的内容。

In [17]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim=512, n_heads=8):
        """
        Args:
            embed_dim:输入维度，MultiHeadAttention 的输入维度必须和 Word Embedding 的输出维度一致
            n_heads:头数，embed_dim 必须能被 n_heads 整除
        """
        super(MultiHeadAttention, self).__init__()

        self.embed_dim = embed_dim
        self.n_heads = heads
        self.single_head_dim = int(self.embed_dim / self.n_heads)

        # 构造 qkv
        self.qiery_matrix = nn.Linear(self.single_head_dim, self.single_head_dim, bias=False)
        self.key_matrix = nn.Linear(self.single_dim, self.single_head_dim, bias=False)
        self.value_amatrix = nn.Linear(self.single_dim, self.single_head_dim, bias=False)
        self.out = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(self, key, query, value, mask=None):
        """
        Args:
            key:key vector
            query:query vector
            value:value vector

        Returns:
            output vector from multihead attention
        """
        batch_size = key.size(0)
        seq_length = key.size(1)

        seq_length_query = query.size(1)

        key = key.view(batch_size, seq_length, self.n_heads, self.single_head_dim)
        
        
        
